In [ ]:
# agents/planner_agent.py
class PlannerAgent:
    def plan(self, filters: dict, fields: list) -> list:
        return [
            "Parse the document",
            "Generate optimized prompt",
            "Extract fields using prompt",
            "Validate result"
        ]


# agents/prompt_builder_agent.py
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

class PromptBuilderAgent:
    def __init__(self):
        self.chain = LLMChain(
            llm=OpenAI(model="gpt-4"),
            prompt=PromptTemplate.from_template(
                """
                You are a prompt engineer for legal documents. Based on filters: {filters} and fields: {fields},
                create a JSON-only prompt for extracting structured data from OCR'd or digital PDF lease documents.
                """
            )
        )

    def run(self, filters: dict, fields: list) -> str:
        return self.chain.run({"filters": str(filters), "fields": str(fields)})


# agents/document_parser_agent.py
from tools.ocr import extract_text_from_pdf

class DocumentParserAgent:
    def parse(self, pdf_path: str) -> dict:
        return extract_text_from_pdf(pdf_path)


# agents/extraction_agent.py
from langchain.llms import OpenAI

class ExtractionAgent:
    def __init__(self):
        self.llm = OpenAI(model="gpt-4")

    def run(self, prompt: str, doc_text: str) -> str:
        full_input = f"{prompt}\n\n### DOCUMENT CONTENT\n{doc_text}"
        return self.llm.invoke(full_input)


# agents/validation_agent.py
from tools.json_validator import validate_json_output

class ValidationAgent:
    def validate(self, raw_output: str) -> tuple:
        try:
            result = json.loads(raw_output)
        except Exception as e:
            return None, f"JSON parsing error: {e}"
        return validate_json_output(result)


# agents/replay_panel.py
import streamlit as st
import os
import json

def render_replay_panel(log_dir="audit_logs"):
    st.sidebar.subheader("🕹️ Agent Replay Panel")
    files = [f for f in os.listdir(log_dir) if f.startswith("prompt")]
    choice = st.sidebar.selectbox("Select Prompt Log", files)
    if choice:
        with open(os.path.join(log_dir, choice), "r") as f:
            prompt = f.read()
        st.code(prompt, language="markdown")
        resp_file = choice.replace("prompt", "response").replace(".txt", ".json")
        with open(os.path.join(log_dir, resp_file), "r") as f:
            st.json(json.load(f))


# agents/claude_tool_agent.py
# Optional Claude-like Tool Agent (mock behavior)
class ClaudeToolAgent:
    def run(self, tool: str, input_text: str) -> str:
        if tool == "summarize":
            return input_text[:300] + "... (summary)"
        elif tool == "extract_dates":
            import re
            return str(re.findall(r"\b\w+ \d{1,2}, \d{4}\b", input_text))
        return "Unknown tool"


# __main__.py to run the pipeline
from agents.planner_agent import PlannerAgent
from agents.prompt_builder_agent import PromptBuilderAgent
from agents.document_parser_agent import DocumentParserAgent
from agents.extraction_agent import ExtractionAgent
from agents.validation_agent import ValidationAgent

class AgentPipeline:
    def __init__(self):
        self.planner = PlannerAgent()
        self.prompt_agent = PromptBuilderAgent()
        self.parser = DocumentParserAgent()
        self.extractor = ExtractionAgent()
        self.validator = ValidationAgent()

    def run(self, pdf_path, filters, fields):
        plan = self.planner.plan(filters, fields)
        parsed = self.parser.parse(pdf_path)
        prompt = self.prompt_agent.run(filters, fields)
        output = self.extractor.run(prompt, parsed['text'])
        validated, err = self.validator.validate(output)
        if err:
            return {"error": err, "raw_output": output}
        return validated.dict()

# Optional UI hook
# from agents.replay_panel import render_replay_panel
# render_replay_panel()


In [ ]:
tools/ocr.py:
def extract_text_from_pdf(pdf_path: str) -> dict:
    try:
        import fitz
        doc = fitz.open(pdf_path)
        return {"text": "\n".join([p.get_text() for p in doc]), "is_ocr": False}
    except:
        from pdf2image import convert_from_path
        import pytesseract
        images = convert_from_path(pdf_path)
        text = " ".join([pytesseract.image_to_string(img) for img in images])
        return {"text": text, "is_ocr": True}


tools/json_validator.py
from pydantic import BaseModel, Field, ValidationError
from typing import Optional, Dict

class FieldExtraction(BaseModel):
    value: Optional[str]
    reasoning: str
    confidence: float = Field(..., ge=0, le=1)

class ExtractionSchema(BaseModel):
    __root__: Dict[str, FieldExtraction]

def validate_json_output(data: dict):
    try:
        return ExtractionSchema.parse_obj(data), None
    except ValidationError as ve:
        return None, str(ve)


In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

class PlannerAgent:
    def __init__(self):
        prompt = PromptTemplate.from_template("""
You are a task planner for an AI system that extracts structured data from scanned or digital lease documents.

Given:
- Fields: {fields}
- Filters: {filters}

Generate a step-by-step plan using available agents:
- DocumentParserAgent
- PromptBuilderAgent
- ExtractionAgent
- ValidationAgent

Respond with a numbered list of agent calls (one per step).
""")
        self.chain = LLMChain(prompt=prompt, llm=OpenAI(model="gpt-4"))

    def generate_plan(self, filters: dict, fields: list) -> list:
        plan_text = self.chain.run(filters=filters, fields=fields)
        return [line.strip().split()[0] for line in plan_text.strip().split("\n") if line.strip()]


In [ ]:

from agents.prompt_builder_agent import PromptBuilderAgent
from agents.document_parser_agent import DocumentParserAgent
from agents.extraction_agent import ExtractionAgent
from agents.validation_agent import ValidationAgent

AGENT_REGISTRY = {
    "DocumentParserAgent": DocumentParserAgent(),
    "PromptBuilderAgent": PromptBuilderAgent(),
    "ExtractionAgent": ExtractionAgent(),
    "ValidationAgent": ValidationAgent(),
}


In [ ]:
from agents.planner_agent import PlannerAgent
from agents.agent_registry import AGENT_REGISTRY

class AgentOrchestrator:
    def __init__(self):
        self.planner = PlannerAgent()

    def run(self, pdf_path, filters, fields):
        context = {
            "pdf_path": pdf_path,
            "filters": filters,
            "fields": fields,
            "intermediates": {}  # stores data shared between agents
        }

        plan = self.planner.generate_plan(filters, fields)

        for agent_name in plan:
            agent = AGENT_REGISTRY.get(agent_name)
            if not agent:
                raise ValueError(f"No agent registered for {agent_name}")

            if agent_name == "DocumentParserAgent":
                context["intermediates"]["parsed"] = agent.parse(pdf_path)

            elif agent_name == "PromptBuilderAgent":
                context["intermediates"]["prompt"] = agent.run(filters, fields)

            elif agent_name == "ExtractionAgent":
                prompt = context["intermediates"]["prompt"]
                text = context["intermediates"]["parsed"]["text"]
                context["intermediates"]["extracted"] = agent.run(prompt, text)

            elif agent_name == "ValidationAgent":
                result = agent.validate(context["intermediates"]["extracted"])
                validated, error = result
                if error:
                    return {"error": error, "raw": context["intermediates"]["extracted"]}
                return validated.dict()

        return context["intermediates"]


In [ ]:
# /agents/memory_board.py
import os
import json

class AgentMemoryBoard:
    def __init__(self, doc_id: str, memory_dir: str = "memory"):
        os.makedirs(memory_dir, exist_ok=True)
        self.path = os.path.join(memory_dir, f"{doc_id}.json")
        self._memory = self._load()

    def _load(self):
        if os.path.exists(self.path):
            with open(self.path, "r") as f:
                return json.load(f)
        return {}

    def save(self):
        with open(self.path, "w") as f:
            json.dump(self._memory, f, indent=2)

    def update(self, key: str, value):
        self._memory[key] = value
        self.save()

    def get(self, key: str):
        return self._memory.get(key)

    def get_all(self):
        return self._memory


# /agents/agent_registry.py
from agents.prompt_builder_agent import PromptBuilderAgent
from agents.document_parser_agent import DocumentParserAgent
from agents.extraction_agent import ExtractionAgent
from agents.validation_agent import ValidationAgent

AGENT_REGISTRY = {
    "DocumentParserAgent": DocumentParserAgent(),
    "PromptBuilderAgent": PromptBuilderAgent(),
    "ExtractionAgent": ExtractionAgent(),
    "ValidationAgent": ValidationAgent(),
}


# /agents/planner_agent.py (dynamic planner)
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

class PlannerAgent:
    def __init__(self):
        prompt = PromptTemplate.from_template("""
You are a task planner for an AI system that extracts structured data from scanned or digital lease documents.

Given:
- Fields: {fields}
- Filters: {filters}

Generate a step-by-step plan using these agents:
- DocumentParserAgent
- PromptBuilderAgent
- ExtractionAgent
- ValidationAgent

Respond with one agent name per line in the order they should be called.
""")
        self.chain = LLMChain(prompt=prompt, llm=OpenAI(model="gpt-4"))

    def generate_plan(self, filters: dict, fields: list) -> list:
        plan_text = self.chain.run(filters=filters, fields=fields)
        return [line.strip() for line in plan_text.strip().split("\n") if line.strip()]


# /agents/orchestrator.py
from agents.planner_agent import PlannerAgent
from agents.agent_registry import AGENT_REGISTRY
from agents.memory_board import AgentMemoryBoard

class AgentOrchestrator:
    def __init__(self, doc_id: str):
        self.memory = AgentMemoryBoard(doc_id)
        self.planner = PlannerAgent()

    def run(self, pdf_path, filters, fields):
        plan = self.planner.generate_plan(filters, fields)

        for agent_name in plan:
            agent = AGENT_REGISTRY.get(agent_name)
            if not agent:
                raise ValueError(f"No agent registered for {agent_name}")

            if agent_name == "DocumentParserAgent":
                parsed = agent.parse(pdf_path)
                self.memory.update("parsed", parsed)

            elif agent_name == "PromptBuilderAgent":
                prompt = agent.run(filters, fields)
                self.memory.update("prompt", prompt)

            elif agent_name == "ExtractionAgent":
                prompt = self.memory.get("prompt")
                text = self.memory.get("parsed")["text"]
                result = agent.run(prompt, text)
                self.memory.update("extracted", result)

            elif agent_name == "ValidationAgent":
                raw = self.memory.get("extracted")
                validated, error = agent.validate(raw)
                self.memory.update("validated", validated.dict() if validated else None)
                if error:
                    self.memory.update("error", error)
                    return {"error": error, "raw": raw}
                return validated.dict()

        return self.memory.get_all()


In [ ]:
# agent_dashboard.py
import streamlit as st
import os
import json
from agents.orchestrator import AgentOrchestrator
from agents.memory_board import AgentMemoryBoard

st.set_page_config(page_title="Agentic Document Intelligence", layout="wide")
st.title("🧠 Agentic Lease Document Analyzer")

# --- Sidebar Config ---
st.sidebar.header("Document & Metadata")
filters = {
    "Module Name": st.sidebar.text_input("Module Name", "Real Estate"),
    "Territory": st.sidebar.text_input("Territory", "South"),
    "Owner": st.sidebar.text_input("Owner", "Crown"),
    "Market": st.sidebar.text_input("Market", "South East"),
    "Contract Type": st.sidebar.text_input("Contract Type", "Lease"),
    "SubMarket": st.sidebar.text_input("SubMarket", "Florida"),
    "Document Type": st.sidebar.text_input("Document Type", "Lease"),
    "Local Market": st.sidebar.text_input("Local Market", "Tampa"),
    "Facility Type": st.sidebar.text_input("Facility Type", "Easement")
}

st.sidebar.markdown("---")

# --- Field Input ---
st.sidebar.header("Fields to Extract")
fields = []
field_count = st.sidebar.slider("# Fields", 1, 10, 3)
for i in range(field_count):
    name = st.sidebar.text_input(f"Field {i+1} Name", key=f"fname_{i}")
    desc = st.sidebar.text_input(f"Field {i+1} Description", key=f"fdesc_{i}")
    if name:
        fields.append({"name": name, "description": desc})

# --- File Upload ---
st.header("Upload PDF Lease Document")
uploaded = st.file_uploader("Choose PDF", type="pdf")

doc_id = None
if uploaded:
    os.makedirs("uploads", exist_ok=True)
    doc_id = uploaded.name.replace(".pdf", "")
    doc_path = os.path.join("uploads", uploaded.name)
    with open(doc_path, "wb") as f:
        f.write(uploaded.read())

    # Run agentic pipeline
    if st.button("Run Agent Pipeline"):
        st.subheader("🧠 Agent Execution Log")
        with st.spinner("Running full agent workflow..."):
            pipeline = AgentOrchestrator(doc_id=doc_id)
            results = pipeline.run(doc_path, filters, fields)
            st.success("Pipeline execution complete ✅")
            st.json(results)

# --- Memory Visualizer ---
if doc_id and os.path.exists(f"memory/{doc_id}.json"):
    st.markdown("---")
    st.header("🗂️ Agent Memory Replay")
    mem = AgentMemoryBoard(doc_id)
    mem_data = mem.get_all()
    for k, v in mem_data.items():
        with st.expander(f"{k}"):
            if isinstance(v, str):
                st.code(v)
            else:
                st.json(v)


In [ ]:
# /agents/prompt_builder_agent.py with retry logic
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

class PromptBuilderAgent:
    def __init__(self, retries: int = 2):
        self.retries = retries
        self.chain = LLMChain(
            llm=OpenAI(model="gpt-4"),
            prompt=PromptTemplate.from_template(
                """
                You are a prompt engineer for legal documents. Based on filters: {filters} and fields: {fields},
                create a JSON-only prompt for extracting structured data from OCR'd or digital PDF lease documents.
                """
            )
        )

    def run(self, filters: dict, fields: list) -> str:
        for attempt in range(self.retries):
            try:
                return self.chain.run({"filters": str(filters), "fields": str(fields)})
            except Exception as e:
                if attempt == self.retries - 1:
                    raise RuntimeError(f"PromptBuilder failed after {self.retries} attempts: {e}")


# /agents/extraction_agent.py with retry logic
from langchain.llms import OpenAI
import time

class ExtractionAgent:
    def __init__(self, retries: int = 2):
        self.retries = retries
        self.llm = OpenAI(model="gpt-4")

    def run(self, prompt: str, doc_text: str) -> str:
        full_input = f"{prompt}\n\n### DOCUMENT CONTENT\n{doc_text}"
        for attempt in range(self.retries):
            try:
                return self.llm.invoke(full_input)
            except Exception as e:
                time.sleep(1)
                if attempt == self.retries - 1:
                    raise RuntimeError(f"ExtractionAgent failed: {e}")


# /agents/validation_agent.py with retry logic
from tools.json_validator import validate_json_output
import json

class ValidationAgent:
    def __init__(self, retries: int = 1):
        self.retries = retries

    def validate(self, raw_output: str) -> tuple:
        for _ in range(self.retries):
            try:
                data = json.loads(raw_output)
                return validate_json_output(data)
            except Exception as e:
                continue
        return None, "Validation failed after retries."


# /agents/memory_flow_panel.py - trace visualizer
import streamlit as st
import os
import json
import datetime
import hashlib

def render_memory_flow(doc_id: str):
    mem_path = os.path.join("memory", f"{doc_id}.json")
    if not os.path.exists(mem_path):
        st.warning("No memory found for this document.")
        return

    with open(mem_path, "r") as f:
        data = json.load(f)

    st.header("🧠 Memory Trace Flow")
    keys = list(data.keys())
    for i in range(len(keys) - 1):
        st.markdown(f"**{keys[i]} ➜ {keys[i+1]}**")
        with st.expander(keys[i]):
            st.json(data[keys[i]])
    if keys:
        with st.expander(keys[-1]):
            st.json(data[keys[-1]])

In [ ]:
# /agents/memory_board.py with metadata + checkpoint replay
import os
import json
from datetime import datetime

class AgentMemoryBoard:
    def __init__(self, doc_id: str, memory_dir: str = "memory"):
        os.makedirs(memory_dir, exist_ok=True)
        self.path = os.path.join(memory_dir, f"{doc_id}.json")
        self._memory = self._load()
        if "_meta" not in self._memory:
            self._memory["_meta"] = {"created": datetime.utcnow().isoformat(), "log": []}

    def _load(self):
        if os.path.exists(self.path):
            with open(self.path, "r") as f:
                return json.load(f)
        return {}

    def save(self):
        with open(self.path, "w") as f:
            json.dump(self._memory, f, indent=2)

    def update(self, key: str, value, retries: int = 0, runtime: float = 0.0):
        self._memory[key] = value
        self._memory["_meta"]["log"].append({
            "step": key,
            "timestamp": datetime.utcnow().isoformat(),
            "retries": retries,
            "runtime": runtime
        })
        self.save()

    def get(self, key: str):
        return self._memory.get(key)

    def get_all(self):
        return self._memory

    def replay_from(self, agent_key: str):
        return {k: self._memory[k] for k in self._memory if k == agent_key or k == "_meta"}


# /agents/memory_flow_panel.py - with Mermaid diagram + replay
import streamlit as st
import os
import json
import datetime


def render_memory_flow(doc_id: str):
    mem_path = os.path.join("memory", f"{doc_id}.json")
    if not os.path.exists(mem_path):
        st.warning("No memory found for this document.")
        return

    with open(mem_path, "r") as f:
        data = json.load(f)

    st.header("📈 Agent Execution Flow")
    if "_meta" not in data or "log" not in data["_meta"]:
        st.warning("No execution log found.")
        return

    logs = data["_meta"]["log"]
    steps = [entry["step"] for entry in logs]
    mermaid = ["flowchart TD"]
    for i in range(len(steps) - 1):
        mermaid.append(f"    {steps[i]} --> {steps[i+1]}")
    st.markdown("```mermaid\n" + "\n".join(mermaid) + "\n```")

    st.markdown("---")
    st.subheader("📂 Step Detail + Replay")
    for entry in logs:
        key = entry["step"]
        with st.expander(f"{key} (retried {entry['retries']}x, runtime: {entry['runtime']}s)"):
            st.json(data.get(key))
            if st.button(f"🔁 Replay {key}", key=f"replay_{key}"):
                st.session_state["replay_step"] = key
                st.success(f"Set to replay from {key}")

In [ ]:
# memory_flow_panel.py with replay resume button
import streamlit as st
import os
import json


def render_memory_flow(doc_id: str):
    mem_path = os.path.join("memory", f"{doc_id}.json")
    if not os.path.exists(mem_path):
        st.warning("No memory found for this document.")
        return

    with open(mem_path, "r") as f:
        data = json.load(f)

    st.header("📈 Agent Execution Flow")
    if "_meta" not in data or "log" not in data["_meta"]:
        st.warning("No execution log found.")
        return

    logs = data["_meta"]["log"]
    steps = [entry["step"] for entry in logs]
    mermaid = ["flowchart TD"]
    for i in range(len(steps) - 1):
        mermaid.append(f"    {steps[i]} --> {steps[i+1]}")
    st.markdown("```mermaid\n" + "\n".join(mermaid) + "\n```")

    st.markdown("---")
    st.subheader("📂 Step Detail + Replay")
    replay_step = st.session_state.get("replay_step")
    resume_button = None

    for entry in logs:
        key = entry["step"]
        with st.expander(f"{key} (retried {entry['retries']}x, runtime: {entry['runtime']}s)"):
            st.json(data.get(key))
            if st.button(f"🔁 Replay {key}", key=f"replay_{key}"):
                st.session_state["replay_step"] = key
                st.success(f"Set to replay from {key}")

    if replay_step:
        st.markdown(f"### ▶️ Ready to Resume from: `{replay_step}`")
        with st.form("resume_form"):
            updated_fields = st.text_area("Optional: Update Field JSON", "")
            run = st.form_submit_button("Resume Pipeline")
            if run:
                if updated_fields:
                    try:
                        fields = json.loads(updated_fields)
                        st.session_state["override_fields"] = fields
                    except json.JSONDecodeError:
                        st.error("Invalid JSON - cannot resume")
                        return
                st.session_state["resume"] = True
                st.success(f"Resuming from {replay_step} with updated fields")



In [ ]:
# /agents/orchestrator.py with resume logic + memory filter override
from agents.planner_agent import PlannerAgent
from agents.agent_registry import AGENT_REGISTRY
from agents.memory_board import AgentMemoryBoard
import time

class AgentOrchestrator:
    def __init__(self, doc_id: str):
        self.memory = AgentMemoryBoard(doc_id)
        self.planner = PlannerAgent()

    def run(self, pdf_path, filters, fields):
        override_fields = None
        replay_point = None

        # Resume mode
        if "resume" in globals() or "resume" in locals():
            override_fields = globals().get("override_fields") or fields
            replay_point = globals().get("replay_step")

        plan = self.planner.generate_plan(filters, override_fields or fields)

        execution_started = False
        for agent_name in plan:
            if replay_point and not execution_started:
                if agent_name == replay_point:
                    execution_started = True
                else:
                    continue

            agent = AGENT_REGISTRY.get(agent_name)
            if not agent:
                raise ValueError(f"No agent registered for {agent_name}")

            t0 = time.time()
            if agent_name == "DocumentParserAgent":
                parsed = agent.parse(pdf_path)
                self.memory.update("parsed", parsed, retries=0, runtime=time.time() - t0)

            elif agent_name == "PromptBuilderAgent":
                prompt = agent.run(filters, override_fields or fields)
                self.memory.update("prompt", prompt, retries=agent.retries, runtime=time.time() - t0)

            elif agent_name == "ExtractionAgent":
                prompt = self.memory.get("prompt")
                text = self.memory.get("parsed")["text"]
                result = agent.run(prompt, text)
                self.memory.update("extracted", result, retries=agent.retries, runtime=time.time() - t0)

            elif agent_name == "ValidationAgent":
                raw = self.memory.get("extracted")
                validated, error = agent.validate(raw)
                self.memory.update("validated", validated.dict() if validated else None, retries=agent.retries, runtime=time.time() - t0)
                if error:
                    self.memory.update("error", error)
                    return {"error": error, "raw": raw}
                return validated.dict()

        return self.memory.get_all()


# /agents/insight_utils.py for confidence trend + PDF export
import matplotlib.pyplot as plt
import json
import os
from fpdf import FPDF


def plot_confidence_trends(doc_id):
    path = f"memory/{doc_id}.json"
    if not os.path.exists(path):
        return None
    with open(path, "r") as f:
        mem = json.load(f)

    validations = mem.get("validated", {})
    confidences = {k: v.get("confidence") for k, v in validations.items() if isinstance(v, dict)}
    if not confidences:
        return None

    plt.figure(figsize=(10, 4))
    plt.bar(confidences.keys(), confidences.values(), color='green')
    plt.title("Confidence Scores for Extracted Fields")
    plt.ylabel("Confidence")
    plt.ylim(0, 1)
    plt.xticks(rotation=45)
    fig_path = f"memory/{doc_id}_confidence.png"
    plt.tight_layout()
    plt.savefig(fig_path)
    return fig_path


def generate_audit_pdf(doc_id):
    mem_path = f"memory/{doc_id}.json"
    img_path = f"memory/{doc_id}_confidence.png"
    if not os.path.exists(mem_path):
        return None

    with open(mem_path, "r") as f:
        data = json.load(f)

    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="Agent Flow Audit Report", ln=True, align='C')
    pdf.ln(10)
    pdf.cell(200, 10, txt=f"Document ID: {doc_id}", ln=True)

    for k in data:
        if k == "_meta": continue
        pdf.set_font("Arial", 'B', 12)
        pdf.cell(200, 10, txt=f"Step: {k}", ln=True)
        pdf.set_font("Arial", size=10)
        txt = json.dumps(data[k], indent=2)
        for line in txt.split("\n"):
            pdf.multi_cell(0, 5, txt=line)
        pdf.ln()

    if os.path.exists(img_path):
        pdf.add_page()
        pdf.image(img_path, x=10, y=30, w=180)

    final_path = f"memory/{doc_id}_audit.pdf"
    pdf.output(final_path)
    return final_path


In [ ]:
# agent_dashboard.py — Add PDF export and confidence trend buttons
import streamlit as st
import os
import json
from agents.orchestrator import AgentOrchestrator
from agents.memory_board import AgentMemoryBoard
from agents.insight_utils import plot_confidence_trends, generate_audit_pdf

st.set_page_config(page_title="Agentic Document Intelligence", layout="wide")
st.title("🧠 Agentic Lease Document Analyzer")

# --- Sidebar Config ---
st.sidebar.header("Document & Metadata")
filters = {
    "Module Name": st.sidebar.text_input("Module Name", "Real Estate"),
    "Territory": st.sidebar.text_input("Territory", "South"),
    "Owner": st.sidebar.text_input("Owner", "Crown"),
    "Market": st.sidebar.text_input("Market", "South East"),
    "Contract Type": st.sidebar.text_input("Contract Type", "Lease"),
    "SubMarket": st.sidebar.text_input("SubMarket", "Florida"),
    "Document Type": st.sidebar.text_input("Document Type", "Lease"),
    "Local Market": st.sidebar.text_input("Local Market", "Tampa"),
    "Facility Type": st.sidebar.text_input("Facility Type", "Easement")
}

st.sidebar.markdown("---")

# --- Field Input ---
st.sidebar.header("Fields to Extract")
fields = []
field_count = st.sidebar.slider("# Fields", 1, 10, 3)
for i in range(field_count):
    name = st.sidebar.text_input(f"Field {i+1} Name", key=f"fname_{i}")
    desc = st.sidebar.text_input(f"Field {i+1} Description", key=f"fdesc_{i}")
    if name:
        fields.append({"name": name, "description": desc})

# --- File Upload ---
st.header("Upload PDF Lease Document")
uploaded = st.file_uploader("Choose PDF", type="pdf")

doc_id = None
if uploaded:
    os.makedirs("uploads", exist_ok=True)
    doc_id = uploaded.name.replace(".pdf", "")
    doc_path = os.path.join("uploads", uploaded.name)
    with open(doc_path, "wb") as f:
        f.write(uploaded.read())

    # Run agentic pipeline
    if st.button("Run Agent Pipeline"):
        st.subheader("🧠 Agent Execution Log")
        with st.spinner("Running full agent workflow..."):
            pipeline = AgentOrchestrator(doc_id=doc_id)
            results = pipeline.run(doc_path, filters, fields)
            st.success("Pipeline execution complete ✅")
            st.json(results)

# --- Memory Visualizer ---
if doc_id and os.path.exists(f"memory/{doc_id}.json"):
    st.markdown("---")
    st.header("🗂️ Agent Memory Replay")
    mem = AgentMemoryBoard(doc_id)
    mem_data = mem.get_all()
    for k, v in mem_data.items():
        with st.expander(f"{k}"):
            if isinstance(v, str):
                st.code(v)
            else:
                st.json(v)

    # Confidence Chart
    st.markdown("---")
    st.subheader("📊 Confidence Score Trends")
    chart_path = plot_confidence_trends(doc_id)
    if chart_path:
        st.image(chart_path, caption="Field-level Confidence Scores")
    else:
        st.info("No valid confidence scores to show.")

    # PDF Export
    st.markdown("---")
    st.subheader("📄 Download Audit Report")
    audit_pdf = generate_audit_pdf(doc_id)
    if audit_pdf:
        with open(audit_pdf, "rb") as f:
            st.download_button("📥 Download Audit PDF", f, file_name=os.path.basename(audit_pdf), mime="application/pdf")


In [ ]:
# agents/chunk_retrieval_agent.py
from retrieval_pipeline import ChunkRetriever, FieldTaskPlanner, RetrievalExecutor
from agents.memory_board import AgentMemoryBoard
from langchain.llms import OpenAI

class ChunkRetrievalAgent:
    def __init__(self, doc_id: str, faiss_index="faiss_index"):
        self.doc_id = doc_id
        self.memory = AgentMemoryBoard(doc_id)
        self.retriever = ChunkRetriever(faiss_index)
        self.llm = OpenAI(model="gpt-4")
        self.executor = RetrievalExecutor(self.retriever, self.llm)

    def run(self, fields):
        planner = FieldTaskPlanner()
        tasks = planner.plan(fields)
        result = self.executor.run_all(tasks)
        self.memory.update("chunk_retrieval_results", result)
        return result


In [ ]:
# orchestrator.py with ChunkRetrievalAgent added
from agents.planner_agent import PlannerAgent
from agents.agent_registry import AGENT_REGISTRY
from agents.memory_board import AgentMemoryBoard
from agents.chunk_retrieval_agent import ChunkRetrievalAgent
import time

class AgentOrchestrator:
    def __init__(self, doc_id: str):
        self.doc_id = doc_id
        self.memory = AgentMemoryBoard(doc_id)
        self.planner = PlannerAgent()
        self.chunk_agent = ChunkRetrievalAgent(doc_id)

    def run(self, pdf_path, filters, fields):
        plan = self.planner.generate_plan(filters, fields)

        for agent_name in plan:
            agent = AGENT_REGISTRY.get(agent_name)
            t0 = time.time()

            if agent_name == "DocumentParserAgent":
                output = agent.parse(pdf_path)
                self.memory.update("parsed", output, runtime=time.time() - t0)

            elif agent_name == "PromptBuilderAgent":
                output = agent.run(filters, fields)
                self.memory.update("prompt", output, retries=agent.retries, runtime=time.time() - t0)

            elif agent_name == "ExtractionAgent":
                prompt = self.memory.get("prompt")
                text = self.memory.get("parsed")["text"]
                output = agent.run(prompt, text)
                self.memory.update("extracted", output, retries=agent.retries, runtime=time.time() - t0)

            elif agent_name == "ChunkRetrievalAgent":
                output = self.chunk_agent.run(fields)
                self.memory.update("chunk_retrieval_results", output, runtime=time.time() - t0)

            elif agent_name == "ValidationAgent":
                raw = self.memory.get("extracted")
                validated, error = agent.validate(raw)
                self.memory.update("validated", validated.dict() if validated else None, runtime=time.time() - t0)
                if error:
                    self.memory.update("error", error)
                    return {"error": error, "raw": raw}
                return validated.dict()

        return self.memory.get_all()


In [ ]:
# memory_flow_panel.py — visual chunk view + replay control
import streamlit as st
import os
import json

def render_memory_flow(doc_id: str):
    mem_path = os.path.join("memory", f"{doc_id}.json")
    if not os.path.exists(mem_path):
        st.warning("No memory found for this document.")
        return

    with open(mem_path, "r") as f:
        data = json.load(f)

    st.header("📈 Agent Execution Trace")
    logs = data.get("_meta", {}).get("log", [])
    steps = [entry["step"] for entry in logs]
    mermaid = ["flowchart TD"]
    for i in range(len(steps) - 1):
        mermaid.append(f"    {steps[i]} --> {steps[i+1]}")
    st.markdown("```mermaid\n" + "\n".join(mermaid) + "\n```")

    st.subheader("🧠 Agent Outputs + Replay")
    for entry in logs:
        key = entry["step"]
        val = data.get(key)
        with st.expander(f"{key} (retried {entry['retries']}x, time: {entry['runtime']:.2f}s)"):
            if key == "chunk_retrieval_results":
                for field, res in val.items():
                    with st.expander(f"🔎 Field: {field}"):
                        st.json(res)
                        if st.button(f"🔁 Retry Field: {field}", key=f"retry_{field}"):
                            st.session_state["replay_chunk_field"] = field
                            st.success(f"Field {field} set for replay")
            else:
                st.json(val)

    if "replay_chunk_field" in st.session_state:
        field = st.session_state["replay_chunk_field"]
        st.markdown(f"### ⏩ Ready to rerun chunk retrieval for: `{field}`")
        if st.button("🔁 Run Chunk Subtask"):
            st.session_state["resume"] = True
            st.session_state["resume_field"] = field
            st.success(f"Triggering LLM + chunk rerun for {field}")


In [ ]:
# memory_flow_panel.py — now with chunk highlights and metadata
import streamlit as st
import os
import json


def render_memory_flow(doc_id: str):
    mem_path = os.path.join("memory", f"{doc_id}.json")
    if not os.path.exists(mem_path):
        st.warning("No memory found for this document.")
        return

    with open(mem_path, "r") as f:
        data = json.load(f)

    st.header("📈 Agent Execution Trace")
    logs = data.get("_meta", {}).get("log", [])
    steps = [entry["step"] for entry in logs]
    mermaid = ["flowchart TD"]
    for i in range(len(steps) - 1):
        mermaid.append(f"    {steps[i]} --> {steps[i+1]}")
    st.markdown("```mermaid\n" + "\n".join(mermaid) + "\n```")

    st.subheader("🧠 Agent Outputs + Replay")
    for entry in logs:
        key = entry["step"]
        val = data.get(key)
        with st.expander(f"{key} (retried {entry['retries']}x, time: {entry['runtime']:.2f}s)"):
            if key == "chunk_retrieval_results":
                for field, res in val.items():
                    with st.expander(f"🔎 Field: {field}"):
                        st.markdown(f"**Value:** `{res.get('value')}`")
                        st.markdown(f"**Reason:** {res.get('reason')}")
                        st.markdown(f"**Confidence:** `{res.get('confidence')}`")

                        # Optional alignment and metadata if present
                        if "_aligned_chunks" in res:
                            for chunk in res["_aligned_chunks"]:
                                st.markdown(f"**Chunk ID:** `{chunk.get('chunk_id')}` | **Page:** `{chunk.get('page_num')}` | **Doc ID:** `{chunk.get('doc_id')}`")
                                content = chunk.get("text", "")
                                highlight = res.get("value", "")
                                if highlight and highlight in content:
                                    content = content.replace(highlight, f"**:green[{highlight}]**")
                                st.markdown(f"> {content}", unsafe_allow_html=True)

                        if st.button(f"🔁 Retry Field: {field}", key=f"retry_{field}"):
                            st.session_state["replay_chunk_field"] = field
                            st.success(f"Field {field} set for replay")
            else:
                st.json(val)

    if "replay_chunk_field" in st.session_state:
        field = st.session_state["replay_chunk_field"]
        st.markdown(f"### ⏩ Ready to rerun chunk retrieval for: `{field}`")
        if st.button("🔁 Run Chunk Subtask"):
            st.session_state["resume"] = True
            st.session_state["resume_field"] = field
            st.success(f"Triggering LLM + chunk rerun for {field}")


In [ ]:
# memory_flow_panel.py — now with chunk highlights and metadata
import streamlit as st
import os
import json


def render_memory_flow(doc_id: str):
    mem_path = os.path.join("memory", f"{doc_id}.json")
    if not os.path.exists(mem_path):
        st.warning("No memory found for this document.")
        return

    with open(mem_path, "r") as f:
        data = json.load(f)

    st.header("📈 Agent Execution Trace")
    logs = data.get("_meta", {}).get("log", [])
    steps = [entry["step"] for entry in logs]
    mermaid = ["flowchart TD"]
    for i in range(len(steps) - 1):
        mermaid.append(f"    {steps[i]} --> {steps[i+1]}")
    st.markdown("```mermaid\n" + "\n".join(mermaid) + "\n```")

    st.subheader("🧠 Agent Outputs + Replay")
    for entry in logs:
        key = entry["step"]
        val = data.get(key)
        with st.expander(f"{key} (retried {entry['retries']}x, time: {entry['runtime']:.2f}s)"):
            if key == "chunk_retrieval_results":
                for field, res in val.items():
                    with st.expander(f"🔎 Field: {field}"):
                        st.markdown(f"**Value:** `{res.get('value')}`")
                        st.markdown(f"**Reason:** {res.get('reason')}")
                        st.markdown(f"**Confidence:** `{res.get('confidence')}`")

                        # Optional alignment and metadata if present
                        if "_aligned_chunks" in res:
                            for chunk in res["_aligned_chunks"]:
                                st.markdown(f"**Chunk ID:** `{chunk.get('chunk_id')}` | **Page:** `{chunk.get('page_num')}` | **Doc ID:** `{chunk.get('doc_id')}`")
                                content = chunk.get("text", "")
                                highlight = res.get("value", "")
                                if highlight and highlight in content:
                                    content = content.replace(highlight, f"**:green[{highlight}]**")
                                st.markdown(f"> {content}", unsafe_allow_html=True)

                        if st.button(f"🔁 Retry Field: {field}", key=f"retry_{field}"):
                            st.session_state["replay_chunk_field"] = field
                            st.success(f"Field {field} set for replay")
            else:
                st.json(val)

    if "replay_chunk_field" in st.session_state:
        field = st.session_state["replay_chunk_field"]
        st.markdown(f"### ⏩ Ready to rerun chunk retrieval for: `{field}`")
        if st.button("🔁 Run Chunk Subtask"):
            st.session_state["resume"] = True
            st.session_state["resume_field"] = field
            st.success(f"Triggering LLM + chunk rerun for {field}")


In [ ]:
# insight_utils.py — include chunk reasoning in audit PDF
import matplotlib.pyplot as plt
import json
import os
from fpdf import FPDF


def plot_confidence_trends(doc_id):
    path = f"memory/{doc_id}.json"
    if not os.path.exists(path):
        return None
    with open(path, "r") as f:
        mem = json.load(f)

    validations = mem.get("validated") or mem.get("chunk_retrieval_results", {})
    confidences = {k: v.get("confidence") for k, v in validations.items() if isinstance(v, dict)}
    if not confidences:
        return None

    plt.figure(figsize=(10, 4))
    plt.bar(confidences.keys(), confidences.values(), color='green')
    plt.title("Confidence Scores for Extracted Fields")
    plt.ylabel("Confidence")
    plt.ylim(0, 1)
    plt.xticks(rotation=45)
    fig_path = f"memory/{doc_id}_confidence.png"
    plt.tight_layout()
    plt.savefig(fig_path)
    return fig_path


def generate_audit_pdf(doc_id):
    mem_path = f"memory/{doc_id}.json"
    img_path = f"memory/{doc_id}_confidence.png"
    if not os.path.exists(mem_path):
        return None

    with open(mem_path, "r") as f:
        data = json.load(f)

    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="Agent Flow Audit Report", ln=True, align='C')
    pdf.ln(10)
    pdf.cell(200, 10, txt=f"Document ID: {doc_id}", ln=True)

    for k in data:
        if k == "_meta": continue
        pdf.set_font("Arial", 'B', 12)
        pdf.cell(200, 10, txt=f"Step: {k}", ln=True)
        pdf.set_font("Arial", size=10)

        if k == "chunk_retrieval_results":
            for field, result in data[k].items():
                pdf.set_font("Arial", 'B', 11)
                pdf.cell(200, 10, txt=f"Field: {field}", ln=True)
                pdf.set_font("Arial", size=9)
                pdf.multi_cell(0, 5, txt=json.dumps({k: v for k, v in result.items() if not k.startswith("_aligned")}, indent=2))
                if "_aligned_chunks" in result:
                    for chunk in result["_aligned_chunks"]:
                        pdf.set_font("Arial", 'I', 9)
                        pdf.multi_cell(0, 5, txt=f"Chunk (Page {chunk['page_num']}, ID {chunk['chunk_id']}):\n{chunk['text'][:300]}...\n")
        else:
            txt = json.dumps(data[k], indent=2)
            for line in txt.split("\n"):
                pdf.multi_cell(0, 5, txt=line)
        pdf.ln()

    if os.path.exists(img_path):
        pdf.add_page()
        pdf.image(img_path, x=10, y=30, w=180)

    final_path = f"memory/{doc_id}_audit.pdf"
    pdf.output(final_path)
    return final_path


In [ ]:
[
  {
    "subtask": "Find the monthly rent amount",
    "type_filter": "table",
    "hints": ["monthly rent", "$"]
  },
  {
    "subtask": "Find the total lease duration in months or years",
    "type_filter": "narrative_text",
    "hints": ["term", "commence", "expire"]
  },
  {
    "subtask": "Multiply rent × duration to calculate total",
    "type_filter": "inferred",
    "calculation_required": true
  }
]


In [ ]:
You are a financial reasoning assistant analyzing lease contracts.

Context:
{{ merged_chunks }}

Task:
Based on the context above, answer the following multi-hop field:
"{{ field_name }}"

Steps:
1. Identify related values (e.g., rent amount, lease term)
2. Infer final value using reasoning or math
3. Return structured JSON

Respond in:
{
  "value": "...",
  "reason": "...",
  "confidence": 0.0–1.0,
  "evidence": [ { "text": "...", "chunk_id": "...", "page": ... } ]
}


In [ ]:
from agents.multi_hop_reasoning_agent import MultiHopReasoningAgent

AGENT_REGISTRY = {
  ...
  "MultiHopReasoningAgent": MultiHopReasoningAgent()
}
if task.get("multi_hop"):
    agent = AGENT_REGISTRY["MultiHopReasoningAgent"]
    output = agent.run(task, self.memory)
    self.memory.update(f"multi_hop_{task['name']}", output)


In [ ]:
# agents/multi_hop_reasoning_agent.py
from retrieval_pipeline import ChunkRetriever
from langchain.llms import OpenAI
import json

class MultiHopSubtaskPlanner:
    def plan_subtasks(self, field_task):
        name = field_task["name"]
        description = field_task.get("description", name)
        hints = field_task.get("hints", [])

        if "total rent" in name.lower():
            return [
                {"subtask": "Find monthly rent", "hints": ["monthly", "$"], "type_filter": "table"},
                {"subtask": "Find lease term duration", "hints": ["term", "years", "months"], "type_filter": "narrative_text"},
                {"subtask": "Calculate total rent = monthly × duration", "inferred": True}
            ]
        else:
            return [
                {"subtask": f"Reason and extract: {description}", "hints": hints}
            ]


class MultiHopReasoningAgent:
    def __init__(self, index_path="faiss_index"):
        self.retriever = ChunkRetriever(index_path)
        self.llm = OpenAI(model="gpt-4")
        self.planner = MultiHopSubtaskPlanner()

    def run(self, field_task, memory):
        subtasks = self.planner.plan_subtasks(field_task)
        all_chunks = []
        merged_context = []

        for sub in subtasks:
            if sub.get("inferred"):
                continue  # Handle below
            retrieved = self.retriever.hybrid_retrieve(
                field_name=sub["subtask"],
                description=sub["subtask"],
                type_filter=sub.get("type_filter"),
                k=5,
                hints=sub.get("hints", [])
            )
            for chunk in retrieved:
                merged_context.append(chunk.page_content)
                all_chunks.append({
                    "chunk_id": chunk.metadata["chunk_id"],
                    "page": chunk.metadata["page"],
                    "type": chunk.metadata["type"],
                    "text": chunk.page_content
                })

        prompt = f"""
You are performing multi-hop reasoning to answer: {field_task['name']}

Context:
{chr(10).join(merged_context)}

Follow these steps:
- Identify partial values from above
- Perform reasoning to calculate or infer final value

Return in JSON:
{{"value": ..., "reason": ..., "confidence": 0.0–1.0, "evidence": [...]}}
"""

        try:
            response = self.llm.invoke(prompt)
            result = json.loads(response)
            result["_trace"] = subtasks
            result["_evidence"] = all_chunks
            return result
        except Exception as e:
            return {"value": None, "reason": f"Multi-hop failed: {str(e)}", "confidence": 0.0}


In [ ]:
# orchestrator.py — integrate MultiHopReasoningAgent and persist trace
from agents.planner_agent import PlannerAgent
from agents.agent_registry import AGENT_REGISTRY
from agents.memory_board import AgentMemoryBoard
from agents.chunk_retrieval_agent import ChunkRetrievalAgent
from agents.multi_hop_reasoning_agent import MultiHopReasoningAgent
import time

class AgentOrchestrator:
    def __init__(self, doc_id: str):
        self.doc_id = doc_id
        self.memory = AgentMemoryBoard(doc_id)
        self.planner = PlannerAgent()
        self.chunk_agent = ChunkRetrievalAgent(doc_id)
        self.multi_hop_agent = MultiHopReasoningAgent()

    def run(self, pdf_path, filters, fields):
        plan = self.planner.generate_plan(filters, fields)

        for agent_name in plan:
            agent = AGENT_REGISTRY.get(agent_name)
            t0 = time.time()

            if agent_name == "DocumentParserAgent":
                output = agent.parse(pdf_path)
                self.memory.update("parsed", output, runtime=time.time() - t0)

            elif agent_name == "PromptBuilderAgent":
                output = agent.run(filters, fields)
                self.memory.update("prompt", output, retries=agent.retries, runtime=time.time() - t0)

            elif agent_name == "ChunkRetrievalAgent":
                output = self.chunk_agent.run(fields)
                self.memory.update("chunk_retrieval_results", output, runtime=time.time() - t0)

            elif agent_name == "ExtractionAgent":
                prompt = self.memory.get("prompt")
                text = self.memory.get("parsed")["text"]
                output = agent.run(prompt, text)
                self.memory.update("extracted", output, retries=agent.retries, runtime=time.time() - t0)

            elif agent_name == "ValidationAgent":
                raw = self.memory.get("extracted")
                validated, error = agent.validate(raw)
                self.memory.update("validated", validated.dict() if validated else None, runtime=time.time() - t0)
                if error:
                    self.memory.update("error", error)
                    return {"error": error, "raw": raw}
                return validated.dict()

        # Post-plan: route multi-hop fields separately
        for field in fields:
            if field.get("multi_hop"):
                result = self.multi_hop_agent.run(field, self.memory)
                key = f"multi_hop_{field['name'].replace(' ', '_')}"
                self.memory.update(key, result, runtime=0)

        return self.memory.get_all()


In [ ]:
# agents/multi_hop_reasoning_agent.py — add hop-level scoring and retry fallback
from retrieval_pipeline import ChunkRetriever
from langchain.llms import OpenAI
import json

class MultiHopSubtaskPlanner:
    def plan_subtasks(self, field_task):
        name = field_task["name"]
        description = field_task.get("description", name)
        hints = field_task.get("hints", [])

        if "total rent" in name.lower():
            return [
                {"subtask": "Find monthly rent", "hints": ["monthly", "$"], "type_filter": "table"},
                {"subtask": "Find lease term duration", "hints": ["term", "years", "months"], "type_filter": "narrative_text"},
                {"subtask": "Calculate total rent = monthly × duration", "inferred": True}
            ]
        else:
            return [
                {"subtask": f"Reason and extract: {description}", "hints": hints}
            ]


class MultiHopReasoningAgent:
    def __init__(self, index_path="faiss_index"):
        self.retriever = ChunkRetriever(index_path)
        self.llm = OpenAI(model="gpt-4")
        self.planner = MultiHopSubtaskPlanner()

    def run(self, field_task, memory):
        subtasks = self.planner.plan_subtasks(field_task)
        all_chunks = []
        merged_context = []
        hops_failed = 0

        for sub in subtasks:
            if sub.get("inferred"):
                continue
            retrieved = self.retriever.hybrid_retrieve(
                field_name=sub["subtask"],
                description=sub["subtask"],
                type_filter=sub.get("type_filter"),
                k=5,
                hints=sub.get("hints", [])
            )
            if not retrieved:
                hops_failed += 1
            for chunk in retrieved:
                merged_context.append(chunk.page_content)
                all_chunks.append({
                    "chunk_id": chunk.metadata["chunk_id"],
                    "page": chunk.metadata["page"],
                    "type": chunk.metadata["type"],
                    "text": chunk.page_content
                })

        prompt = f"""
You are performing multi-hop reasoning to answer: {field_task['name']}

Context:
{chr(10).join(merged_context)}

Follow these steps:
- Identify partial values from above
- Perform reasoning to calculate or infer final value

Return in JSON:
{{"value": ..., "reason": ..., "confidence": 0.0–1.0, "evidence": [...]}}
"""

        try:
            response = self.llm.invoke(prompt)
            result = json.loads(response)
            result["_trace"] = subtasks
            result["_evidence"] = all_chunks
            result["_hops_failed"] = hops_failed
            if hops_failed >= len(subtasks) - 1:
                result["confidence"] = 0.0
                result["reason"] = result.get("reason", "Too many subtasks failed to support reasoning")
            return result
        except Exception as e:
            return {
                "value": None,
                "reason": f"Multi-hop failed: {str(e)}",
                "confidence": 0.0,
                "_trace": subtasks,
                "_evidence": all_chunks,
                "_hops_failed": hops_failed
            }


In [ ]:
# agents/multi_hop_reasoning_agent.py — fallback to direct extraction on total hop failure
from retrieval_pipeline import ChunkRetriever, RetrievalExecutor
from langchain.llms import OpenAI
import json

class MultiHopSubtaskPlanner:
    def plan_subtasks(self, field_task):
        name = field_task["name"]
        description = field_task.get("description", name)
        hints = field_task.get("hints", [])

        if "total rent" in name.lower():
            return [
                {"subtask": "Find monthly rent", "hints": ["monthly", "$"], "type_filter": "table"},
                {"subtask": "Find lease term duration", "hints": ["term", "years", "months"], "type_filter": "narrative_text"},
                {"subtask": "Calculate total rent = monthly × duration", "inferred": True}
            ]
        else:
            return [
                {"subtask": f"Reason and extract: {description}", "hints": hints}
            ]


class MultiHopReasoningAgent:
    def __init__(self, index_path="faiss_index"):
        self.retriever = ChunkRetriever(index_path)
        self.llm = OpenAI(model="gpt-4")
        self.executor = RetrievalExecutor(self.retriever, self.llm)
        self.planner = MultiHopSubtaskPlanner()

    def run(self, field_task, memory):
        subtasks = self.planner.plan_subtasks(field_task)
        all_chunks = []
        merged_context = []
        hops_failed = 0

        for sub in subtasks:
            if sub.get("inferred"):
                continue
            retrieved = self.retriever.hybrid_retrieve(
                field_name=sub["subtask"],
                description=sub["subtask"],
                type_filter=sub.get("type_filter"),
                k=5,
                hints=sub.get("hints", [])
            )
            if not retrieved:
                hops_failed += 1
            for chunk in retrieved:
                merged_context.append(chunk.page_content)
                all_chunks.append({
                    "chunk_id": chunk.metadata["chunk_id"],
                    "page": chunk.metadata["page"],
                    "type": chunk.metadata["type"],
                    "text": chunk.page_content
                })

        # fallback to single-hop if all non-inferred hops fail
        if hops_failed >= len([s for s in subtasks if not s.get("inferred")]):
            fallback = self.executor.run_task(field_task)
            fallback["_trace"] = ["multi-hop fallback: single-hop extraction used"]
            fallback["_hops_failed"] = hops_failed
            fallback["_fallback_used"] = True
            return fallback

        prompt = f"""
You are performing multi-hop reasoning to answer: {field_task['name']}

Context:
{chr(10).join(merged_context)}

Follow these steps:
- Identify partial values from above
- Perform reasoning to calculate or infer final value

Return in JSON:
{{"value": ..., "reason": ..., "confidence": 0.0–1.0, "evidence": [...]}}
"""

        try:
            response = self.llm.invoke(prompt)
            result = json.loads(response)
            result["_trace"] = subtasks
            result["_evidence"] = all_chunks
            result["_hops_failed"] = hops_failed
            result["_fallback_used"] = False
            return result
        except Exception as e:
            return {
                "value": None,
                "reason": f"Multi-hop failed: {str(e)}",
                "confidence": 0.0,
                "_trace": subtasks,
                "_evidence": all_chunks,
                "_hops_failed": hops_failed,
                "_fallback_used": False
            }


In [ ]:
# memory_flow_panel.py — display multi-hop fallback, hops failed, and trace
import streamlit as st
import os
import json

def render_memory_flow(doc_id: str):
    mem_path = os.path.join("memory", f"{doc_id}.json")
    if not os.path.exists(mem_path):
        st.warning("No memory found for this document.")
        return

    with open(mem_path, "r") as f:
        data = json.load(f)

    st.header("📈 Agent Execution Trace")
    logs = data.get("_meta", {}).get("log", [])
    steps = [entry["step"] for entry in logs]
    mermaid = ["flowchart TD"]
    for i in range(len(steps) - 1):
        mermaid.append(f"    {steps[i]} --> {steps[i+1]}")
    st.markdown("```mermaid\n" + "\n".join(mermaid) + "\n```")

    st.subheader("🧠 Agent Outputs + Replay")
    for key, val in data.items():
        if key.startswith("multi_hop_"):
            with st.expander(f"🧠 {key} (Multi-Hop Field Result)"):
                st.markdown(f"**Value:** `{val.get('value')}`")
                st.markdown(f"**Reason:** {val.get('reason')}")
                st.markdown(f"**Confidence:** `{val.get('confidence')}`")
                st.markdown(f"**Fallback Used:** `{val.get('_fallback_used', False)}`")
                st.markdown(f"**Hops Failed:** `{val.get('_hops_failed', 0)}`")

                if "_trace" in val:
                    st.markdown("**Reasoning Trace:**")
                    for step in val["_trace"]:
                        st.code(step if isinstance(step, str) else json.dumps(step, indent=2))

                if "_evidence" in val:
                    st.markdown("**Supporting Evidence Chunks:**")
                    for chunk in val["_evidence"]:
                        with st.expander(f"Chunk {chunk['chunk_id']} (Page {chunk['page']}, Type {chunk['type']})"):
                            st.code(chunk["text"][:800])

    for entry in logs:
        key = entry["step"]
        val = data.get(key)
        if key.startswith("multi_hop_"):
            continue  # already shown
        with st.expander(f"{key} (retried {entry['retries']}x, time: {entry['runtime']:.2f}s)"):
            st.json(val)
